In [1]:
import h3
import geopandas as gpd
import rasterio
from rasterio.enums import Resampling
import rasterstats as rs
from shapely.geometry import Polygon

In [2]:
def reclassify_raster(input_raster, output_raster):
    # Otwieranie pliku rasterowego
    with rasterio.open(input_raster) as src:
        data = src.read(1)  # Wczytanie pierwszej warstwy

        # Reklasyfikacja danych
        data[data == 2] = 0
        data[data == 1] = 1  # Zamiana wartości 2 na 1

        # Zapis nowego pliku z zaktualizowanymi danymi
        with rasterio.open(
            output_raster,
            'w',
            driver='GTiff',
            height=data.shape[0],
            width=data.shape[1],
            count=1,
            dtype=data.dtype,
            crs=src.crs,
            transform=src.transform,
        ) as dst:
            dst.write(data, 1)


def zonal_stats(raster_path, hex_gdf, field):   

    with rasterio.open(raster_path) as raster_interpolate:
        # Masked = True sets no data values to np.nan if they are in the metadata
        raster_data = raster_interpolate.read(1, masked=True)
        raster_meta = raster_interpolate.profile

    hexagon_zonal = rs.zonal_stats(hex_gdf,
                                    raster_data,
                                    nodata=-999,
                                    affine=raster_meta['transform'],
                                    geojson_out=True,
                                    copy_properties=True,
                                    stats="count min mean max median")

    hexagons_zonal = gpd.GeoDataFrame.from_features(hexagon_zonal)
    hexagons_zonal[field] = hexagons_zonal['mean']
    # hexagons_zonal = hexagons_zonal[['geometry', field]]
    return hexagons_zonal



def load_gdf_and_generate_h3(file_path, resolution=9):
    # Wczytanie GeoDataFrame
    gdf = gpd.read_file(file_path)

    # Lista na przechowywanie heksagonów H3
    hexagons = []

    # Przetwarzanie każdego poligonu w GDF
    for index, row in gdf.iterrows():
        # Uzyskanie geometrii każdego poligonu
        poly = row['geometry']

        # Generowanie heksagonów H3 dla danego poligonu
        hex_ids = h3.polyfill(poly.__geo_interface__, resolution, geo_json_conformant=True)

        # Tworzenie poligonów z heksagonów
        hex_polygons = [Polygon(h3.h3_to_geo_boundary(h, geo_json=True)) for h in hex_ids]

        # Dodawanie do listy
        hexagons.extend(hex_polygons)

    # Tworzenie nowego GeoDataFrame
    hex_gdf = gpd.GeoDataFrame(geometry=hexagons)

    return hex_gdf

In [3]:
# Przykład użycia
file_path = r'd:\GIS\GIS_projects\2024_jakarta\johor\shps\johor_study_area.shp'
hex_gdf = load_gdf_and_generate_h3(file_path)
print(hex_gdf)

                                                geometry
0      POLYGON ((103.54675 1.55744, 103.54629 1.55931...
1      POLYGON ((103.61522 1.36083, 103.61477 1.36269...
2      POLYGON ((103.85246 1.60377, 103.85201 1.60564...
3      POLYGON ((103.58112 1.62293, 103.58066 1.62480...
4      POLYGON ((103.61753 1.50131, 103.61708 1.50317...
...                                                  ...
11384  POLYGON ((103.91889 1.54358, 103.91844 1.54545...
11385  POLYGON ((103.99981 1.50206, 103.99936 1.50392...
11386  POLYGON ((103.54119 1.48750, 103.54074 1.48937...
11387  POLYGON ((103.57499 1.68393, 103.57454 1.68580...
11388  POLYGON ((103.66589 1.57337, 103.66544 1.57524...

[11389 rows x 1 columns]


In [4]:
# Przykład użycia funkcji reklas
input_raster_path = r'd:\GIS\GIS_projects\2024_jakarta\johor\2_classes\2024_20042024_11classes.tif'
output_raster_path_2024 = r'd:\GIS\GIS_projects\2024_jakarta\johor\2_classes\2024_20042024_11classes_rekl.tif'
reclassify_raster(input_raster_path, output_raster_path_2024)

input_raster_path = r'd:\GIS\GIS_projects\2024_jakarta\johor\2_classes\2016_21042024_11classes.tif'
output_raster_path_2019 = r'd:\GIS\GIS_projects\2024_jakarta\johor\2_classes\2016_21042024_11classes_rekl.tif'
reclassify_raster(input_raster_path, output_raster_path_2019)

In [5]:
# zonals
hexes = zonal_stats(output_raster_path_2019, hex_gdf, '2016')
hexes = zonal_stats(output_raster_path_2024, hexes, '2024')

In [6]:
hexagons_zonal = hexes[['geometry', '2016', '2024']]


In [7]:
# # TODO time series analysis
# przepusci wzrost tylko wtedy jesli mamy wiecej niz 10 pp
# albo wiecje niz 50% zmiany
# albo wiecej niz 50% ale nie mniej niz 10 pp

In [8]:
hexes

,geometry,min,max,mean,count,median,2016,2024
0,"POLYGON ((103.54675 1.55744, 103.54629 1.55931...",0.0,0.0,0.000000,1201,0.0,0.000000,0.000000
1,"POLYGON ((103.61522 1.36083, 103.61477 1.36269...",0.0,1.0,0.041667,1200,0.0,0.038333,0.041667
2,"POLYGON ((103.85246 1.60377, 103.85201 1.60564...",0.0,0.0,0.000000,1204,0.0,0.000000,0.000000
3,"POLYGON ((103.58112 1.62293, 103.58066 1.62480...",0.0,1.0,0.040563,1208,0.0,0.057947,0.040563
4,"POLYGON ((103.61753 1.50131, 103.61708 1.50317...",0.0,1.0,0.437604,1202,0.0,0.470882,0.437604
...,...,...,...,...,...,...,...,...
11384,"POLYGON ((103.91889 1.54358, 103.91844 1.54545...",0.0,0.0,0.000000,1200,0.0,0.000000,0.000000
11385,"POLYGON ((103.99981 1.50206, 103.99936 1.50392...",0.0,0.0,0.000000,1199,0.0,0.000000,0.000000
11386,"POLYGON ((103.54119 1.48750, 103.54074 1.48937...",0.0,1.0,0.026534,1206,0.0,0.348259,0.026534
11387,"POLYGON ((103.57499 1.68393, 103.57454 1.68580...",0.0,1.0,0.043947,1206,0.0,0.004146,0.043947


In [9]:
# Funkcja, która sprawdzi warunki
def calculate_new_column(row):
    if row['2024'] >= row['2016'] * 1.2 and row['2024'] >= row['2016'] + 0.1:
        return row['2024']
    else:
        return row['2016']

# Stworzenie nowej kolumny
hexes['2024_v2'] = hexes.apply(calculate_new_column, axis=1)

# TODOS
# greenfield - dodatkowa kategoria- od zera jak jest cokolwiek pewnego

In [10]:
hexes['change'] = hexes['2024_v2'] -  hexes['2016']  

In [13]:
hexes.to_file(r'd:\GIS\GIS_projects\2024_jakarta\johor\h3\h3_2016_2024.shp')

In [ ]:
# TODO
# greenfield - dodatkowa kategoria- od zera jak jest cokolwiek pewnego
# zliczanie powierzchni excavation YoY